In [1]:
import plotly.express as px
import pandas as pd
from datetime import datetime as dt, timedelta

In [2]:
receptura_produktu = [
    {"nazwa": "Prod_1",
     "kategoria": "Produkt koncowy",
     "jednostka": "kpl",
     "skladowe": [{
                    "nazwa": "Karton maly", 
                    "ilosc": 1,
                    "jednostka": "kpl"
     },
                   {
                        "nazwa": "Szklanka Late 200ml z uszkiem fi5mm",
                        "ilosc": 6,
                        "jednostka": "szt"
                    }]},
    {
     "nazwa": "Szklanka Late 200ml z uszkiem fi5mm",
     "kategoria": "Asortyment",
     "jednostka": "szt",
     "skladowe" : [{"nazwa": "Szklanka Late 200ml",
                    "ilosc": 1,
                    "jednostka": "szt"},
                    {"nazwa": "uszko 5mm",
                     "ilosc": 1,
                      "jednostka": "szt" }]
                },
   
    {
     "nazwa": "Karton maly",
     "kategoria": "pol produkt",
     "jednostka": "kpl",
     "skladowe" : [{"nazwa": "karton xs",
                    "ilosc": 1,
                    "jednostka": "szt"},
                    {"nazwa": "przekladka dluga",
                     "ilosc": 1,
                      "jednostka": "szt" },
                     
                    {"nazwa": "przekladka krutka",
                     "ilosc": 2,
                      "jednostka": "szt" }]
                },
   
]

In [3]:
zamowienia = [
   dict(Numer_zamowinie= "2023_1", Produkt= "Prod_1", Ilosc=1500, Data_dostawy=dt(2023,6,15,13)),
   # dict(Numer_zamowinie= "Zam1", Produkt= "Prod_2", Ilosc=3000, Data_dostawy=dt(2023,5,15)),
   dict(Numer_zamowinie= "2023_2", Produkt= "Prod_1", Ilosc=10000, Data_dostawy=dt(2023,7,15,13)),
   # dict(Numer_zamowinie= "Zam3", Produkt= "Prod_2", Ilosc=5000, Data_dostawy=dt(2023,5,15)),
   dict(Numer_zamowinie= "2023_3", Produkt= "Prod_1", Ilosc=3000, Data_dostawy=dt(2023,9,15,13)),
] 
zamowienia

[{'Numer_zamowinie': '2023_1',
  'Produkt': 'Prod_1',
  'Ilosc': 1500,
  'Data_dostawy': datetime.datetime(2023, 6, 15, 13, 0)},
 {'Numer_zamowinie': '2023_2',
  'Produkt': 'Prod_1',
  'Ilosc': 10000,
  'Data_dostawy': datetime.datetime(2023, 7, 15, 13, 0)},
 {'Numer_zamowinie': '2023_3',
  'Produkt': 'Prod_1',
  'Ilosc': 3000,
  'Data_dostawy': datetime.datetime(2023, 9, 15, 13, 0)}]

In [4]:
zam_prod1 = 1500 #klp

for s in receptura_produktu[0]["skladowe"]:
    print(s["nazwa"], s["ilosc"]*zam_prod1, s["jednostka"])
    # for 

Karton maly 1500 kpl
Szklanka Late 200ml z uszkiem fi5mm 9000 szt


In [36]:
tehnologia = [
    {"Nazwa":"Prod_1","idop":100, "operacja_nazwa": "przegotowanie form", "norma": "65min/10000szt", "asortyment powiazany": "Asor_1","id_wejsciowe":0, "opuznienie":0},
    {"Nazwa":"Prod_1","idop":200, "operacja_nazwa": "formowanie reczne", "norma": "100szt/h", "asortyment powiazany": "Asor_1","id_wejsciowe":100, "opuznienie":130},
    {"Nazwa":"Prod_1","idop":300, "operacja_nazwa": "giecie preta", "norma": "400szt/h", "asortyment powiazany": "Asor_1","id_wejsciowe":0, "opuznienie":0},
    {"Nazwa":"Prod_1","idop":400, "operacja_nazwa": "klenie uszek reczne", "norma": "120szt/h", "asortyment powiazany": "Asor_1","id_wejsciowe":[200,300], "opuznienie":0},
    {"Nazwa":"Prod_1","idop":500, "operacja_nazwa": "ciecie kartonu", "norma": "130kpl/h", "asortyment powiazany": "Prod_1","id_wejsciowe":0, "opuznienie":0},
    {"Nazwa":"Prod_1","idop":600, "operacja_nazwa": "pakowanie", "norma": "250kpl/h", "asortyment powiazany": "Prod_1","id_wejsciowe":[400,500], "opuznienie":0},
    {"Nazwa":"Prod_1","idop":700, "operacja_nazwa": "zaladunek", "norma": "1h", "asortyment powiazany": "Prod_1","id_wejsciowe":600, "opuznienie":0},
    
    ]
pd.DataFrame(tehnologia)

,Nazwa,idop,operacja_nazwa,norma,asortyment powiazany,id_wejsciowe,opuznienie
0,Prod_1,100,przegotowanie form,65min/10000szt,Asor_1,0,0
1,Prod_1,200,formowanie reczne,100szt/h,Asor_1,100,130
2,Prod_1,300,giecie preta,400szt/h,Asor_1,0,0
3,Prod_1,400,klenie uszek reczne,120szt/h,Asor_1,"[200, 300]",0
4,Prod_1,500,ciecie kartonu,130kpl/h,Prod_1,0,0
5,Prod_1,600,pakowanie,250kpl/h,Prod_1,"[400, 500]",0
6,Prod_1,700,zaladunek,1h,Prod_1,600,0


In [37]:
9000/120/3


25.0

In [54]:
zam1 = "2023_1"
zal1 = [zamowienia[0]["Data_dostawy"]-timedelta(days=3)- timedelta(minutes=60), zamowienia[0]["Data_dostawy"]-timedelta(3)]
pak1 = [zal1[0]-timedelta(hours=(zamowienia[0]["Ilosc"]/250)),zal1[0]]
cie_k1 = [pak1[0] - timedelta(hours=(zamowienia[0]["Ilosc"]/130)) ,pak1[0]]
klu_r1 = [pak1[0]-timedelta(hours=25),pak1[0]]
klu_r2 = [pak1[0]-timedelta(hours=25),pak1[0]]
klu_r3 = [pak1[0]-timedelta(hours=25),pak1[0]]

to = [
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="zaladunek", stanowisko="zaladunek_1", 
        Start=zal1[0], End=zal1[1],Ilosc=""),
     
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="pakowanie", stanowisko="pakowanie h1s1", 
        Start=pak1[0], End=pak1[1],Ilosc="1500kpl"),
    
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="ciecie kartonu", stanowisko="ciecie kartonu", 
        Start=cie_k1[0], End=cie_k1[1],Ilosc="1500kpl"),
        
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="klejenie uszek reczne", stanowisko="klejnie uszek r1", 
        Start=klu_r1[0], End=klu_r1[1],Ilosc="3000szt"),
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="klejenie uszek reczne", stanowisko="klejnie uszek r2", 
        Start=klu_r1[0], End=klu_r1[1],Ilosc="3000szt"),
   dict(Zamowienie=zam1, Produkt="Prod_1", Operacja="klejenie uszek reczne", stanowisko="klejnie uszek r3", 
        Start=klu_r1[0], End=klu_r1[1],Ilosc="3000szt"),
    ]

tabela_operacje = pd.DataFrame(to)

tabela_operacje["Czas operacji"] = tabela_operacje.End-tabela_operacje.Start
tabela_operacje.sort_values(by=["Operacja","Start"])
tabela_operacje

,Zamowienie,Produkt,Operacja,stanowisko,Start,End,Ilosc,Czas operacji
0,2023_1,Prod_1,zaladunek,zaladunek_1,2023-06-12 12:00:00.000000,2023-06-12 13:00:00,,0 days 01:00:00
1,2023_1,Prod_1,pakowanie,pakowanie h1s1,2023-06-12 06:00:00.000000,2023-06-12 12:00:00,1500kpl,0 days 06:00:00
2,2023_1,Prod_1,ciecie kartonu,ciecie kartonu,2023-06-11 18:27:41.538462,2023-06-12 06:00:00,1500kpl,0 days 11:32:18.461538
3,2023_1,Prod_1,klejenie uszek reczne,klejnie uszek r1,2023-06-11 05:00:00.000000,2023-06-12 06:00:00,3000szt,1 days 01:00:00
4,2023_1,Prod_1,klejenie uszek reczne,klejnie uszek r2,2023-06-11 05:00:00.000000,2023-06-12 06:00:00,3000szt,1 days 01:00:00
5,2023_1,Prod_1,klejenie uszek reczne,klejnie uszek r3,2023-06-11 05:00:00.000000,2023-06-12 06:00:00,3000szt,1 days 01:00:00


In [55]:
tabela_operacje.iloc[2]["Czas operacji"]

Timedelta('0 days 11:32:18.461538')

In [53]:
fig = px.timeline(tabela_operacje, x_start="Start", x_end="End", y="stanowisko", color="Zamowienie",title="Operacje", hover_data=["Produkt", "Czas operacji", "Ilosc"])
# fig.update_yaxes(autorange="reversed")
fig.show()